In [95]:
import torch
import json
import cv2
from datetime import date
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ibug.face_detection import RetinaFacePredictor
from ibug.face_parsing import FaceParser as RTNetPredictor
%matplotlib inline

In [96]:
threshold = 0.6 # default = 0.8
weights = None # r"C:\mahmoud_dev\machine learning\segmentation\face_parsing\ibug\face_parsing\rtnet\weights\rtnet101-fcn-14.torch" # default = None
num_classes = 14 # default = 11
max_num_faces = 50 # default = 50

parser_encoder = 'rtnet101'
parser_decoder = 'fcn'
rotate_image = False
save_json_file = True

today = date.today()


if torch.cuda.is_available():
    device = 'cuda:0'
face_detector = RetinaFacePredictor(threshold=threshold, device=device, model=(RetinaFacePredictor.get_model('mobilenet0.25')))
face_parser = RTNetPredictor(device=device, ckpt=weights, encoder=parser_encoder, decoder=parser_decoder, num_classes=num_classes)



Hybrid stages [True, True, True]


In [97]:
categories_list = ['background', 'skin', 'left_eyebrow', 'right_eyebrow', 'left_eye', 'right_eye',
                 'nose', 'upper_lip', 'inner_mouth', 'lower_lip', 'hair']
new_categories = ['background', 'skin', 'eyebrow', 'eye', 'hair', 'glasses', 'beard']

if num_classes==14:
    categories_list.extend(['left_ear', 'right_ear',  'glasses', 'beard'])

categories = [{"name":category, "id":i, "type": "any", "attributes":[]} for i, category in enumerate(new_categories)]

print(categories)
# Initialize COCO format dictionary
coco_dict = {
    'info': {"contributor":"mahmoud_tabikh","date_created":date.today().strftime('%d-%m-%Y'),"description":"", "version":""},
    'licenses': [],
    'categories': categories,
    'images': [],
    'annotations': []
}

[{'name': 'background', 'id': 0, 'type': 'any', 'attributes': []}, {'name': 'skin', 'id': 1, 'type': 'any', 'attributes': []}, {'name': 'eyebrow', 'id': 2, 'type': 'any', 'attributes': []}, {'name': 'eye', 'id': 3, 'type': 'any', 'attributes': []}, {'name': 'hair', 'id': 4, 'type': 'any', 'attributes': []}, {'name': 'glasses', 'id': 5, 'type': 'any', 'attributes': []}, {'name': 'beard', 'id': 6, 'type': 'any', 'attributes': []}]


In [98]:

def convert_class_id(class_id):
    new_categories = ['background', 'skin', 'eyebrow', 'eye', 'hair', 'glasses', 'beard']
    og_category_dict = {category:id_ for id_, category in enumerate(categories_list)}
    new_category_dict = {category:id_ for id_, category in enumerate(new_categories)}

    to_skin = [og_category_dict[value] for value in ["nose", "upper_lip", "inner_mouth", "lower_lip", "left_ear", "right_ear"]]
    to_single_brow = [og_category_dict[value] for value in ["left_eyebrow", "right_eyebrow"]]
    to_single_eye = [og_category_dict[value] for value in ["left_eye", "right_eye"]]
    identical_names = {og_category_dict["skin"]:new_category_dict["skin"],
                       og_category_dict["hair"]:new_category_dict["hair"],
                       og_category_dict["glasses"]:new_category_dict["glasses"]}

    if class_id in to_skin:
        return new_category_dict["skin"]
    elif class_id in to_single_brow:
        return new_category_dict["eyebrow"]
    elif class_id in to_single_eye:
        return new_category_dict["eye"]
    else:
        return identical_names[class_id]

In [99]:
# different segmentation methods

def get_segmenation_xy(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    ys, xs = np.where(mask_bool)
    segmentation = np.asarray(list(zip(xs, ys))).flatten().tolist()
    return area, segmentation

def get_segmentation_countours(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    retrieval_method = cv2.RETR_EXTERNAL # options: cv2.RETR_EXTERNAL, cv2.RETR_TREE
    contour_approximation = cv2.CHAIN_APPROX_SIMPLE # options: cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(mask_bool.astype(np.uint8), retrieval_method, contour_approximation)
    segmentations = []
    for contour in contours:
        if len(contour) < 3:
            continue
        segmentation = []
        for point in contour:
            segmentation.extend(point.flatten().tolist())
        segmentations.append(segmentation)
    return area, segmentations


def get_segmentation_countours_test(mask, class_id):
    # Create segmentation and bbox arrays
    mask_bool = mask == class_id
    area = np.sum(mask_bool)

    retrieval_method = cv2.RETR_EXTERNAL # options: cv2.RETR_EXTERNAL, cv2.RETR_TREE
    contour_approximation = cv2.CHAIN_APPROX_SIMPLE # options: cv2.CHAIN_APPROX_SIMPLE, cv2.CHAIN_APPROX_NONE
    contours, hierarchy = cv2.findContours(mask_bool.astype(np.uint8), retrieval_method, contour_approximation)
    segmentations = []
    skin_classid = 1 # in trinamix new labels dict

    if class_id != skin_classid:
        for contour in contours:
            if len(contour) < 3:
                continue
            segmentation = []
            for point in contour:
                segmentation.extend(point.flatten().tolist())
            segmentations.append(segmentation)
    else:
        segmentations = [list(contour.flatten()) for contour in contours if len(contour) > 3]
        if len(segmentations)>1:
            segmentations = [[item for sublist in segmentations for item in sublist]]
    return area, segmentations



In [100]:
def save_segmentation_coco(image_dir):
    non_labelled = []
    # Loop through images in directory
    for image_id, filename in enumerate(tqdm((os.listdir(image_dir)))):
        if filename.endswith(tuple([".png", ".jpg"])):
            image_path = os.path.join(image_dir, filename)
            image_id+=1
            image = cv2.imread(image_path)
            image_dict = { 
                'id': image_id,
                'width': image.shape[1],
                'height': image.shape[0],
                'file_name': filename}
            coco_dict['images'].append(image_dict)
            try:

                faces, masks = get_image_pred(image, face_detector, face_parser, filename)
                mask, face = masks[0], faces[0] # assumes 1 face per image, loop for more faces.
                annotations_list = get_annotations_list(mask, face, image_id, coco_dict["annotations"])
                for dict_ in annotations_list:
                    coco_dict['annotations'].append(dict_)
            except RuntimeError as e:
                non_labelled.append(get_image_info(filename))

    if save_json_file:
        save_json(coco_dict)

    return sorted(non_labelled)


def get_image_pred(img, face_detector, face_parser, filename):
    used_df = False
    if rotate_image:
        img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    faces = face_detector(img)
    masks = face_parser.predict_img(img, faces)
    return faces, masks

def get_annotations_list(mask, face, image_id, coco_annotation_dict):
    # Create annotation dictionary for each unique mask value
    annotation_list = [] # i am making a list of lists instead of list of dicts
    og_category_dict = {category:id_ for id_, category in enumerate(categories_list)}
    for class_id in np.unique(mask):
        if class_id not in [og_category_dict[category] for category in ["background", "nose", "left_ear", "right_ear", "upper_lip", "inner_mouth", "lower_lip"]]:
            annotation_id = len(coco_annotation_dict) + 1
            annotation_dict = {
                'id': annotation_id,
                'image_id': image_id,
                'category_id': convert_class_id(class_id),
                'segmentation': [],
                'bbox': face[:4].astype(int),
                'iscrowd': 0}
            area, segmentations = get_segmentation_countours(mask, class_id)
            annotation_dict['segmentation'].extend(segmentations)
            annotation_dict['area'] = area
            annotation_list.append(annotation_dict)
    # Add annotation to COCO dictionary
    return annotation_list

def save_json(coco_dict):
    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            if isinstance(obj, np.floating):
                return float(obj)
            if isinstance(obj, np.ndarray):
                return obj.tolist()
            return super(NpEncoder, self).default(obj)
    Path(json_filepath).write_text(json.dumps(coco_dict, cls=NpEncoder, indent=3))
    print(f"json file written to {json_filepath}")

def get_image_info(image_filename):
    image_nb = int(image_filename.split(".")[0])
    return image_nb


In [101]:
# Set paths and filenames
image_dir = r'D:\_Xchng\Mahmoud\segmenation\DS01-segmentation_test\data\raw\task_d_765_to_1019'
json_filepath = os.path.join(image_dir, r'instances_default.json')
txt_filepath = os.path.join(image_dir, r'non_labeled.txt')

In [102]:
non_labelled = save_segmentation_coco(image_dir)

100%|██████████| 257/257 [02:00<00:00,  2.13it/s]


json file written to D:\_Xchng\Mahmoud\segmenation\DS01-segmentation_test\data\raw\task_d_765_to_1019\instances_default.json


In [103]:
all_ = len(os.listdir(image_dir))
non_det = len(non_labelled)
frac_non_det = non_det/all_
frac_non_det

0.019455252918287938

In [104]:
with open(txt_filepath, 'w') as f:
    f.write(f"non detected fraction: {frac_non_det}\n")
    for line in sorted(non_labelled):
        f.write(f"{line}\n")